**In this code, we build a WGAN from the ground up to generate new samples that can augment an existing dataset. The process involves defining a generator that creates synthetic data from a latent space and a critic (similar to a discriminator) that distinguishes between real and fake data. The generator and critic are trained iteratively, with the critic aiming to improve its ability to distinguish real from fake samples and the generator trying to create more convincing fake samples.**

In [2]:
from numpy import expand_dims,mean,ones
from numpy.random import randn, randint
from keras.datasets.mnist import load_data
from keras import backend
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Dense,Reshape,Flatten,Conv2D,Conv2DTranspose,LeakyReLU,BatchNormalization
from keras.initializers import RandomNormal
from keras.constraints import Constraint
from matplotlib import pyplot as plt


In [4]:
class ClipConstraint(Constraint):
    # Initialize the constraint with a clipping value
    def __init__(self, clip_value):
        self.clip_value = clip_value

    # The callable method that applies the constraint
    def __call__(self, weights):
        # Clip weights to the hypercube [-clip_value, clip_value]
        return backend.clip(weights, -self.clip_value, self.clip_value)

    # Retrieve configuration settings for serialization or reconstruction
    def get_config(self):
        return {'clip_value': self.clip_value}

# Custom loss function for Wasserstein GANs
def wasserstein_loss(y_true, y_pred):
    # The loss is calculated as the mean of the product of true and predicted labels
    # For Wasserstein GANs, this helps maintain the 1-Lipschitz condition for stable training
    return backend.mean(y_true * y_pred)

# Define the critic model for a Wasserstein GAN
def define_critic(in_shape=(28, 28, 1)):
    # Weight initialization with a small standard deviation for stability
    init = RandomNormal(stddev=0.02)

    # Weight constraint to enforce 1-Lipschitz continuity by clipping weights
    const = ClipConstraint(0.01)

    # Create a sequential model for the critic
    model = Sequential()

    # Add a convolutional layer with downsampling (from 28x28 to 14x14)
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init, kernel_constraint=const, input_shape=in_shape))
    # Batch normalization to stabilize training
    model.add(BatchNormalization())
    # Leaky ReLU activation for non-linear activation
    model.add(LeakyReLU(alpha=0.2))

    # Add another convolutional layer with downsampling (from 14x14 to 7x7)
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init, kernel_constraint=const))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    # Flatten the output for the final dense layer
    model.add(Flatten())

    # Output layer with a single neuron and linear activation for the critic's score
    model.add(Dense(1))

    # Compile the model with RMSprop optimizer and Wasserstein loss
    opt = RMSprop(learning_rate=0.00005)  # RMSprop with a low learning rate for stability
    model.compile(loss=wasserstein_loss, optimizer=opt)

    return model

In [5]:
# Define the generator model for a GAN
def define_generator(latent_dim):
    # Weight initialization with a small standard deviation for stability
    init = RandomNormal(stddev=0.02)

    # Create a sequential model for the generator
    model = Sequential()

    # The generator starts with a dense layer to create the "foundation" for a 7x7x128 image
    n_nodes = 128 * 7 * 7  # 128 filters, 7x7 spatial dimensions
    model.add(Dense(n_nodes, kernel_initializer=init, input_dim=latent_dim))

    # Leaky ReLU activation function for non-linear activation
    model.add(LeakyReLU(alpha=0.2))

    # Reshape the dense output to a 7x7x128 "image"
    model.add(Reshape((7, 7, 128)))

    # Transposed convolution layer to upsample from 7x7 to 14x14
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init))
    model.add(BatchNormalization())  # Normalize to help stabilize training
    model.add(LeakyReLU(alpha=0.2))

    # Another transposed convolution layer to upsample from 14x14 to 28x28
    model.add(Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same', kernel_initializer=init))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    # Output layer to create the final 28x28x1 image with tanh activation (ranges from -1 to 1)
    model.add(Conv2D(1, (7, 7), activation='tanh', padding='same', kernel_initializer=init))

    return model

# Define the combined generator-critic (GAN) model for training the generator
def define_gan(generator, critic):
    # Make critic layers non-trainable to prevent updating them during generator training
    for layer in critic.layers:
        # BatchNormalization layers are typically left trainable for statistical reasons
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False

    # Create a sequential model to combine generator and critic
    model = Sequential()

    # Add the generator to the model
    model.add(generator)

    # Add the critic to the model
    model.add(critic)

    # Compile the combined model with RMSprop optimizer and the Wasserstein loss function
    # RMSprop is used in Wasserstein GANs to maintain stability
    opt = RMSprop(learning_rate=0.00005)  # Low learning rate for stability
    model.compile(loss=wasserstein_loss, optimizer=opt)

    return model

In [6]:
# Function to load and process real image samples from the dataset
def load_real_samples():
    # Load the MNIST dataset
    (trainX, trainy), (_, _) = load_data()  # Load only the training set

    # Select all images of a specific class (e.g., digit '7')
    selected_ix = trainy == 7  # Boolean array indicating where trainy equals 7
    X = trainX[selected_ix]  # Select the corresponding images

    # Expand the dimensions of the images to include the channel dimension (for grayscale)
    X = expand_dims(X, axis=-1)  # Converts shape from (n, 28, 28) to (n, 28, 28, 1)

    # Convert pixel values from integers to floats for further processing
    X = X.astype('float32')  # Change data type to float

    # Scale the pixel values from [0, 255] to the [-1, 1] range (common for GANs with tanh activation)
    X = (X - 127.5) / 127.5

    return X  # Return the processed dataset containing the selected class

# Function to select a specific number of real samples for training/testing
def generate_real_samples(dataset, n_samples):
    # Randomly select indices for the desired number of samples
    ix = randint(0, dataset.shape[0], n_samples)  # Generate random indices

    # Select the corresponding images using the generated indices
    X = dataset[ix]  # Extract the randomly selected images

    # Create class labels for these samples, with -1 indicating 'real' (as used in Wasserstein GANs)
    y = -ones((n_samples, 1))  # Label for real samples

    return X, y  # Return the images and their corresponding labels

# Function to generate random points in the latent space for generator input
def generate_latent_points(latent_dim, n_samples):
    # Generate random points in the latent space (Gaussian distribution)
    x_input = randn(latent_dim * n_samples)  # Generate a 1D array of random numbers

    # Reshape the 1D array into a 2D array of shape (n_samples, latent_dim)
    x_input = x_input.reshape(n_samples, latent_dim)  # Prepare for input to the generator

    return x_input  # Return the generated latent points

# Function to generate fake samples using the generator
def generate_fake_samples(generator, latent_dim, n_samples):
    # Generate random points in the latent space
    x_input = generate_latent_points(latent_dim, n_samples)  # Generate random latent points

    # Use the generator to create fake samples from the latent points
    X = generator.predict(x_input)  # Generate the fake images

    # Create class labels for these samples, with 1 indicating 'fake' (in GANs, fake samples are labeled as 1)
    y = ones((n_samples, 1))  # Label for fake samples

    return X, y  # Return the generated fake samples and their corresponding labels

In [7]:
# Function to generate and save plots of fake samples, and save the generator model
def summarize_performance(step, g_model, latent_dim, n_samples=100):
    # Generate fake samples from the generator
    X, _ = generate_fake_samples(g_model, latent_dim, n_samples)

    # Scale the fake samples from [-1, 1] to [0, 1] for plotting
    X = (X + 1) / 2.0  # Bring the data into a visual-friendly range

    # Plot the generated samples in a 10x10 grid
    plt.figure(figsize=(10, 10))  # Create a new plot
    for i in range(10 * 10):
        # Define the subplot for each image
        plt.subplot(10, 10, 1 + i)  # 10x10 grid
        plt.axis('off')  # Hide axes for better visualization
        # Display the image data in grayscale (inverted for better contrast)
        plt.imshow(X[i, :, :, 0], cmap='gray_r')

    # Save the plot to a file with a name indicating the current step
    filename1 = f'generated_plot_{step + 1:04d}.png'
    plt.savefig(filename1)  # Save the image plot
    plt.close()  # Close the plot

    # Save the current state of the generator model to a file
    filename2 = f'model_{step + 1:04d}.h5'
    g_model.save(filename2)  # Save the model in HDF5 format

    # Inform the user that the plot and model have been saved
    print(f'>Saved: {filename1} and {filename2}')  # Output confirmation to the console

# Function to plot the training loss history for the GAN and save to a file
def plot_history(d1_hist, d2_hist, g_hist):
    # Create a new plot
    plt.figure()

    # Plot the loss history for the critic when judging real samples
    plt.plot(d1_hist, label='Critic (Real)')

    # Plot the loss history for the critic when judging fake samples
    plt.plot(d2_hist, label='Critic (Fake)')

    # Plot the loss history for the generator
    plt.plot(g_hist, label='Generator')

    # Add a legend to differentiate between the plotted lines
    plt.legend()

    # Save the plot to a file
    plt.savefig('plot_line_plot_loss.png')  # Save the line plot

    # Close the plot to free up memory
    plt.close()

In [8]:
# Function to train the GAN, comprising generator and critic models
def train(g_model, c_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=64, n_critic=5):
    # Calculate the number of batches per training epoch
    bat_per_epo = int(dataset.shape[0] / n_batch)  # Number of batches in each epoch

    # Calculate the total number of training steps for all epochs
    n_steps = bat_per_epo * n_epochs  # Total training iterations

    # Calculate the size of half a batch, used for training the critic
    half_batch = int(n_batch / 2)  # Half a batch for real/fake samples

    # Lists to keep track of critic and generator loss history
    c1_hist, c2_hist, g_hist = [], [], []  # To monitor training progress

    # Iterate through each training step
    for i in range(n_steps):
        # Temporary lists to store critic losses for this step
        c1_tmp, c2_tmp = [], []

        # Train the critic more frequently than the generator
        for _ in range(n_critic):
            # Generate random 'real' samples from the dataset
            X_real, y_real = generate_real_samples(dataset, half_batch)  # Real samples
            # Update the critic with real samples, returns the loss
            c_loss1 = c_model.train_on_batch(X_real, y_real)  # Train critic on real samples
            c1_tmp.append(c_loss1)  # Store critic loss on real samples

            # Generate 'fake' examples using the generator
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)  # Fake samples
            # Update the critic with fake samples, returns the loss
            c_loss2 = c_model.train_on_batch(X_fake, y_fake)  # Train critic on fake samples
            c2_tmp.append(c_loss2)  # Store critic loss on fake samples

        # Average the critic losses and append to the history
        c1_hist.append(mean(c1_tmp))  # Average loss on real samples
        c2_hist.append(mean(c2_tmp))  # Average loss on fake samples

        # Prepare random points in the latent space for generator training
        X_gan = generate_latent_points(latent_dim, n_batch)  # Input for the GAN

        # Create labels for the generator's output (-1 for fake, as this is WGAN)
        y_gan = -np.ones((n_batch, 1))  # Inverted labels for GAN training

        # Train the generator via the GAN model (through the critic's feedback)
        g_loss = gan_model.train_on_batch(X_gan, y_gan)  # Generator loss
        g_hist.append(g_loss)  # Store generator loss

        # Print a summary of the current losses
        print(f'> Step {i + 1}, Critic (Real) Loss: {c1_hist[-1]:.3f}, Critic (Fake) Loss: {c2_hist[-1]:.3f}, Generator Loss: {g_loss:.3f}')

        # Summarize model performance every epoch
        if (i + 1) % bat_per_epo == 0:  # At the end of each epoch
            summarize_performance(i, g_model, latent_dim)  # Generate plots and save model

    # Plot loss history to track GAN training progress
    plot_history(c1_hist, c2_hist, g_hist)

# Define the size of the latent space for the generator
latent_dim = 50

# Create the critic (discriminator) model
critic = define_critic()  # Using the function defined earlier

# Create the generator model
generator = define_generator(latent_dim)  # Using the defined generator function

# Create the combined GAN model (generator and critic)
gan_model = define_gan(generator, critic)  # Using the defined GAN function

# Load real image data for training the GAN
dataset = load_real_samples()  # Load and preprocess the dataset
print("Dataset shape:", dataset.shape)  # Output the shape of the dataset

# Start training the GAN
train(generator, critic, gan_model, dataset, latent_dim)  # Train the GAN model

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


11490434/11490434 [==============================] - 1s 0us/step
(6265, 28, 28, 1)
1/1 [==============================] - 0s 169ms/step
>1, c1=-14.116, c2=0.012 g=1.640
1/1 [==============================] - 0s 166ms/step
>2, c1=-29.796, c2=-4.653 g=21.700
1/1 [==============================] - 0s 175ms/step
>3, c1=-40.747, c2=-22.603 g=15.502
1/1 [==============================] - 0s 189ms/step
>4, c1=-52.167, c2=-40.124 g=23.287
1/1 [==============================] - 0s 307ms/step
>5, c1=-65.837, c2=-56.438 g=20.386
1/1 [==============================] - 0s 170ms/step
>6, c1=-72.589, c2=-51.679 g=-16.191
1/1 [==============================] - 0s 183ms/step
>7, c1=-89.955, c2=-81.327 g=-17.847
1/1 [==============================] - 0s 280ms/step
>8, c1=-104.639, c2=-97.890 g=-1.564
1/1 [==============================] - 0s 180ms/step
>9, c1=-117.821, c2=-112.389 g=-20.514
1/1 [==============================] - 0s 188ms/step
>10, c1=-134.380, c2=-127.638 g=-15.920
1/1 [================

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


>Saved: generated_plot_0097.png and model_0097.h5
1/1 [==============================] - 0s 204ms/step
>98, c1=-1778.447, c2=-1927.611 g=-1862.010
1/1 [==============================] - 0s 199ms/step
>99, c1=-1925.676, c2=-2039.754 g=-1663.089
1/1 [==============================] - 0s 286ms/step
>100, c1=-1841.423, c2=-2011.352 g=-1999.355
1/1 [==============================] - 0s 182ms/step
>101, c1=-1948.133, c2=-2036.163 g=-1936.812
1/1 [==============================] - 0s 175ms/step
>102, c1=-2075.974, c2=-2285.315 g=-2119.293
1/1 [==============================] - 0s 188ms/step
>103, c1=-2000.035, c2=-2045.560 g=-2105.720
1/1 [==============================] - 0s 284ms/step
>104, c1=-2119.218, c2=-2233.699 g=-1948.059
1/1 [==============================] - 0s 206ms/step
>105, c1=-2078.819, c2=-2194.162 g=-2198.125
1/1 [==============================] - 0s 236ms/step
>106, c1=-2114.403, c2=-2209.118 g=-2272.631
1/1 [==============================] - 0s 281ms/step
>107, c1=-2170.17

>Saved: generated_plot_0194.png and model_0194.h5
1/1 [==============================] - 0s 241ms/step
>195, c1=-3702.977, c2=3496.653 g=-2516.210
1/1 [==============================] - 0s 181ms/step
>196, c1=-3753.949, c2=3511.415 g=-2312.476
1/1 [==============================] - 0s 363ms/step
>197, c1=-3683.318, c2=3569.425 g=-2578.599
1/1 [==============================] - 0s 184ms/step
>198, c1=-3769.123, c2=3719.347 g=-2924.109
1/1 [==============================] - 0s 189ms/step
>199, c1=-3747.216, c2=3579.857 g=-3354.994
1/1 [==============================] - 0s 369ms/step
>200, c1=-3780.758, c2=3578.966 g=-3366.742
1/1 [==============================] - 0s 177ms/step
>201, c1=-3739.650, c2=3714.277 g=-3533.167
1/1 [==============================] - 0s 196ms/step
>202, c1=-3691.515, c2=3585.293 g=-3698.679
1/1 [==============================] - 0s 224ms/step
>203, c1=-3699.158, c2=3693.564 g=-3686.485
1/1 [==============================] - 0s 191ms/step
>204, c1=-3661.204, c2=3

...

>Saved: generated_plot_0582.png and model_0582.h5
1/1 [==============================] - 0s 173ms/step
>583, c1=-3050.843, c2=3404.756 g=-2813.908
1/1 [==============================] - 0s 187ms/step
>584, c1=-3022.271, c2=3385.258 g=-2875.701
1/1 [==============================] - 0s 197ms/step
>585, c1=-3017.434, c2=3389.264 g=-2848.646
1/1 [==============================] - 0s 179ms/step
>586, c1=-2991.537, c2=3408.136 g=-2957.098
1/1 [==============================] - 0s 330ms/step
>587, c1=-3026.078, c2=3381.075 g=-2918.646
1/1 [==============================] - 0s 181ms/step
>588, c1=-3030.251, c2=3447.703 g=-2807.939
1/1 [==============================] - 0s 176ms/step
>589, c1=-3002.382, c2=3391.316 g=-2889.329
1/1 [==============================] - 0s 193ms/step
>590, c1=-3051.242, c2=3458.157 g=-3008.882
1/1 [==============================] - 0s 180ms/step
>591, c1=-3026.517, c2=3417.283 g=-2993.283
1/1 [==============================] - 0s 177ms/step
>592, c1=-3049.153, c2=3

>Saved: generated_plot_0679.png and model_0679.h5
1/1 [==============================] - 0s 167ms/step
>680, c1=-3387.215, c2=3632.785 g=-3319.819
1/1 [==============================] - 0s 180ms/step
>681, c1=-3402.053, c2=3668.309 g=-3279.896
1/1 [==============================] - 0s 174ms/step
>682, c1=-3376.214, c2=3642.500 g=-3351.626
1/1 [==============================] - 0s 173ms/step
>683, c1=-3374.817, c2=3578.858 g=-3283.644
1/1 [==============================] - 0s 173ms/step
>684, c1=-3370.131, c2=3625.877 g=-3311.947
1/1 [==============================] - 0s 335ms/step
>685, c1=-3394.657, c2=3630.256 g=-3338.326
1/1 [==============================] - 0s 179ms/step
>686, c1=-3411.228, c2=3648.152 g=-3355.702
1/1 [==============================] - 0s 193ms/step
>687, c1=-3435.302, c2=3661.371 g=-3385.437
1/1 [==============================] - 0s 193ms/step
>688, c1=-3413.824, c2=3672.258 g=-3366.421
1/1 [==============================] - 0s 176ms/step
>689, c1=-3357.931, c2=3

>Saved: generated_plot_0776.png and model_0776.h5
1/1 [==============================] - 0s 183ms/step
>777, c1=-3539.762, c2=3745.204 g=-3487.295
1/1 [==============================] - 0s 300ms/step
>778, c1=-3549.983, c2=3735.686 g=-3522.988
1/1 [==============================] - 0s 205ms/step
>779, c1=-3540.019, c2=3730.381 g=-3515.623
1/1 [==============================] - 0s 183ms/step
>780, c1=-3527.421, c2=3716.270 g=-3506.085
1/1 [==============================] - 0s 333ms/step
>781, c1=-3546.854, c2=3742.065 g=-3534.212
1/1 [==============================] - 0s 163ms/step
>782, c1=-3528.272, c2=3736.143 g=-3523.935
1/1 [==============================] - 0s 176ms/step
>783, c1=-3550.771, c2=3752.976 g=-3536.812
1/1 [==============================] - 0s 168ms/step
>784, c1=-3535.676, c2=3740.101 g=-3559.601
1/1 [==============================] - 0s 335ms/step
>785, c1=-3562.260, c2=3740.354 g=-3521.629
1/1 [==============================] - 0s 171ms/step
>786, c1=-3534.344, c2=3

>Saved: generated_plot_0873.png and model_0873.h5
1/1 [==============================] - 0s 177ms/step
>874, c1=-3764.683, c2=3939.945 g=-3764.819
1/1 [==============================] - 0s 274ms/step
>875, c1=-3777.294, c2=3945.000 g=-3736.351
1/1 [==============================] - 0s 192ms/step
>876, c1=-3771.892, c2=3944.221 g=-3703.276
1/1 [==============================] - 0s 181ms/step
>877, c1=-3742.591, c2=3919.856 g=-3710.603
1/1 [==============================] - 0s 314ms/step
>878, c1=-3753.068, c2=3945.345 g=-3718.728
1/1 [==============================] - 0s 200ms/step
>879, c1=-3773.940, c2=3933.043 g=-3743.298
1/1 [==============================] - 0s 191ms/step
>880, c1=-3770.636, c2=3938.297 g=-3746.980
1/1 [==============================] - 0s 203ms/step
>881, c1=-3764.627, c2=3944.208 g=-3739.482
1/1 [==============================] - 0s 173ms/step
>882, c1=-3762.030, c2=3931.549 g=-3720.566
1/1 [==============================] - 0s 217ms/step
>883, c1=-3768.283, c2=3

>Saved: generated_plot_0970.png and model_0970.h5
